In [ ]:
#!/usr/bin/env python3
"""
Fetch the LOT_SIZE.minQty (and related filters) for a given symbol
from Binance's public REST API.
"""

import sys
import requests

BINANCE_API = "https://api.binance.com"
EXCHANGE_INFO = "/api/v3/exchangeInfo"

def get_lot_size_filters(symbol: str) -> dict:
    """
    Retrieve the LOT_SIZE filter for `symbol` from Binance's /exchangeInfo.

    Returns a dict with keys: minQty, maxQty, stepSize (all strings).
    Raises on network errors or if symbol / filter is missing.
    """
    url = f"{BINANCE_API}{EXCHANGE_INFO}?symbol={symbol}"
    resp = requests.get(url, timeout=10)
    resp.raise_for_status()             # throws for HTTP errors
    data = resp.json()

    symbols = data.get("symbols")
    if not symbols:
        raise ValueError(f"No symbol data returned for {symbol!r}")

    filters = symbols[0].get("filters", [])
    for f in filters:
        if f.get("filterType") == "LOT_SIZE":
            return {
                "minQty": f["minQty"],
                "maxQty": f["maxQty"],
                "stepSize": f["stepSize"],
            }

    raise ValueError(f"LOT_SIZE filter not found for {symbol!r}")


def main():
    # You can change this to any symbol you care about
    symbol = "REDUSDC"

    try:
        lot_size = get_lot_size_filters(symbol)
    except Exception as e:
        print(f"Error fetching LOT_SIZE for {symbol}: {e}", file=sys.stderr)
        sys.exit(1)

    print(f"Symbol      : {symbol}")
    print(f"minQty      : {lot_size['minQty']}")
    print(f"maxQty      : {lot_size['maxQty']}")
    print(f"stepSize    : {lot_size['stepSize']}")


if __name__ == "__main__":
    main()
